In [ ]:
# !pip3 install --user --upgrade git+https://github.com/twintproject/twint.git@origin/master#egg=twint

In [ ]:
import twint
import pathlib
import calendar
import datetime
import pandas as pd
import time

# Twint fix
import nest_asyncio
nest_asyncio.apply()

In [ ]:
def info(year, month, day, hashtag, msg):
    now = datetime.datetime.now()
    print(f"[{now.strftime('%Y%m%d-%H%M%S')}] [{year}/{month}/{day}:{hashtag}] {msg}", end='\r')

def mkdir_if_not_exists(dir_name):
    directory = pathlib.Path(dir_name)
    directory.exists()
    directory.mkdir(parents = True, exist_ok = True)
    
def path_generatior(year, month, day, hashtag):
    return f"data/{year}/{month:02d}/{day:02d}/{year}-{month:02d}-{day:02d}-{hashtag.replace('#', '')}.pd"

def collect_tweets(year, month, day, hashtag):
    info(year, month, day, hashtag, "Collecting...")
    
    c = twint.Config()
    
    since = datetime.date(year=year, month=month, day=day)
    until = since + datetime.timedelta(days=1)

    c.Search = hashtag
    c.Since = since.strftime('%Y-%m-%d')
    c.Until = until.strftime('%Y-%m-%d')
    c.Limit = 10000
    c.Retries_count = 10
    # c.Geo = "39.6459604,-99.1227847,1400km"
    # c.Location = True
    c.Hide_output = True
    c.Pandas = True
    
    info(year, month, day, hashtag, "Searching...")
    twint.run.Search(c)
    
    df = twint.storage.panda.Tweets_df
    
    info(year, month, day, hashtag, f"Found {df.shape[0]} tweets")
    
    info(year, month, day, hashtag, "Storing...")
    path = path_generatior(year, month, day, hashtag)
    df.reset_index().to_feather(path)
    
    time.sleep(3)
    
HASHTAGS = [
    '#GlobalWarming',
    '#ClimateChange',
]

mkdir_if_not_exists('data')

def collect_all_tweets():
    for year in range(2021, 2013, -1):
        mkdir_if_not_exists(f"data/{year}")

        for month in range(1, 13):
            mkdir_if_not_exists(f"data/{year}/{month:02d}")

            for day in range(1, calendar.monthrange(year, month)[1]+1, 7):
                mkdir_if_not_exists(f"data/{year}/{month:02d}/{day:02d}")

                for hashtag in HASHTAGS:
                    if pathlib.Path(path_generatior(year, month, day, hashtag)).is_file():
                        continue

                    collect_tweets(year, month, day, hashtag)
        
collect_all_tweets()

In [ ]:
def peek_at_tweets():
    # pd.set_option("display.max_rows", None, "display.max_columns", None)
    
    for year in range(2021, 2013, -1):
        mkdir_if_not_exists(f"data/{year}")

        for month in range(1, 13):
            mkdir_if_not_exists(f"data/{year}/{month:02d}")

            for day in range(1, calendar.monthrange(year, month)[1]+1, 7):
                mkdir_if_not_exists(f"data/{year}/{month:02d}/{day:02d}")

                for hashtag in HASHTAGS:
                    path = path_generatior(year, month, day, hashtag)

                    if not pathlib.Path(path).is_file():
                        continue

                    info(year, month, day, hashtag, f"Peak at {year}/{month}/{day} for {hashtag}")

                    df = pd.read_feather(path)
                    display(df)
                    
peek_at_tweets()